In [19]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sgp4.ext import invjday
from joblib import Parallel, delayed
from sgp4.api import Satrec, jday
from datetime import datetime, timedelta
from concurrent.futures import ProcessPoolExecutor
from skyfield.api import load, wgs84, EarthSatellite

In [20]:
username = 'asas4539@hanyang.ac.kr'
password = 'onsaemiro1729!!'

In [21]:
def get_tle(number):
    # Space-Track API의 URL
    url = "https://www.space-track.org/ajaxauth/login"

    # 요청에 필요한 인증 정보
    payload = {"identity": username, "password": password}

    # Session 객체 생성
    session = requests.Session()

    # 로그인 요청
    response = session.post(url, data=payload)

    if response.status_code != 200:
        print("로그인에 실패했습니다.")
        return None

    # TLE 요청
    response = session.get(
        f"https://www.space-track.org/basicspacedata/query/class/tle/NORAD_CAT_ID/{number}/EPOCH/2022-12-31--2024-07-31/orderby/EPOCH%20asc/format/3le/emptyresult/show"
    )

    return response

In [22]:
def set_plot_params(title, ylabel):
    plt.title(title)
    plt.ylabel(ylabel, fontsize=12)
    plt.xlabel("Date", fontsize=12)
    plt.yticks(fontsize=12)
    plt.xticks(rotation=30, fontsize=12)
    plt.gca().xaxis.set_major_locator(plt.MultipleLocator(30))


def calculate_ylim(data):
    ylim_min = min(data) - (max(data) - min(data)) * 0.1
    ylim_max = max(data) + (max(data) - min(data)) * 0.1
    return ylim_min, ylim_max


def draw_graph(data: pd.DataFrame, title: str, ylabel: str, date_lst: list, color="blue"):
    '''
    :param data: 시간 인덱스와 그래프로 표시할 정보가 있는 데이터프레임
    :param title: 그래프 제목
    :param ylabel: 그래프 y축 단위
    :param date_lst: 아웃라이어 날짜인덱스 리스트
    :param color: 그래프 색깔
    :return: 
    '''
    plt.figure(figsize=(10, 5))
    plt.plot(data.index, data.values, color=color)
    for date in date_lst:
        plt.axvline(pd.to_datetime(date), color='red', linestyle='--', label='Highlight Date')
    set_plot_params(title, ylabel)
    ylim_min, ylim_max = calculate_ylim(list(data.values))
    plt.ylim(ylim_min, ylim_max)
    plt.show()

In [35]:
def process_epoch(epoch, change_df):
    earth_radius = 6378.137
    rad2deg = 180.0 / np.pi
    year = epoch.year
    month = epoch.month
    date = epoch.day
    hour = epoch.hour
    minute = epoch.minute
    second = epoch.second
    print(2)
    now_time = f'{year}-{month}-{date} {hour}:{minute}:{second}'
    temp = change_df[change_df['time'] <= pd.to_datetime(now_time)].copy()
    tle1, tle2 = temp.iloc[-1]['first_line'], temp.iloc[-1]['second_line']

    satellite = Satrec.twoline2rv(tle1, tle2)
    jd, fr = jday(year, month, date, hour, minute, second)
    e, position, velocity = satellite.sgp4(jd, fr)

    # 추가된 부분
    ts = load.timescale()
    t = ts.utc(year, month, date, hour, minute, second)
    earth_satellite = EarthSatellite(tle1, tle2, '', ts)
    geocentric = earth_satellite.at(t)
    subpoint = wgs84.subpoint(geocentric)
    longitude = subpoint.longitude.degrees

    return [
        len(temp), tle1, tle2,
        epoch.year, epoch.month, epoch.day, epoch.hour, epoch.minute, epoch.second,
        *position, *velocity,
        (sum(p ** 2 for p in position) ** 0.5) - earth_radius,
        (sum(v ** 2 for v in velocity) ** 0.5) / earth_radius,
        longitude,
        satellite.alta * earth_radius,
        satellite.altp * earth_radius,
        satellite.inclo * rad2deg, satellite.ecco,
        satellite.nodeo
    ]

def get_info(seg):
    change_df_seg = seg

    start_info = change_df_seg.iloc[0]
    epoch = datetime(start_info['year'], start_info['month'], start_info['day'], start_info['hour'],
                     start_info['minute'], int(start_info['second']))
    epoch += timedelta(minutes=1)

    end_info = change_df_seg.iloc[-1]
    end_epoch = datetime(end_info['year'], end_info['month'], end_info['day'], end_info['hour'], end_info['minute'],
                         int(end_info['second']))


    epochs = []
    while epoch <= end_epoch:
        epochs.append(epoch)
        epoch += timedelta(minutes=1)

    # 병렬 처리
    
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_epoch, epochs, change_df_seg))

    info_df = pd.DataFrame(results)

    info_df.columns = ['change', 'tle1', 'tle2',
                       'year', 'month', 'day', 'hour', 'minute', 'second',
                       'x', 'y', 'z', 'vx', 'vy', 'vz',
                       'altitude', 'velocity', 'longitude',
                       'apogee', 'perigee',
                       'inclination', 'eccentricity', 'raan']

    return info_df

In [36]:
number_lst = [51969, 58722, 56361,
              56289, 56783, 48018, 43823, 45246, 29349, 37265, 42691, 42984, 55841]

to_do_lst = []

In [37]:
for number in number_lst:
    try:
        print(number)
        response = get_tle(number)
        tle_datas = response.text.split('\r\n')

        tle_first_lst = []
        tle_second_lst = []

        for i in range(0, len(tle_datas) - 2, 3):
            tle_first_lst.append(tle_datas[i + 1])
            tle_second_lst.append(tle_datas[i + 2])

        tle_df = pd.DataFrame({'first_line': tle_first_lst, 'second_line': tle_second_lst})
        tle_df.drop_duplicates(keep='first', inplace=True)
        tle_df.reset_index(inplace=True, drop=True)

        change_times = []
        for i in range(len(tle_df)):
            tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['second_line'].iloc[i]
            satellite = Satrec.twoline2rv(tle_1, tle_2)
            jdsatepoch = satellite.jdsatepoch
            jdsatepochfrac = satellite.jdsatepochF
            epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
            change_times.append(epochdatetime)

        change_df = pd.DataFrame(change_times)
        change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
        change_df['first_line'] = tle_df['first_line']
        change_df['second_line'] = tle_df['second_line']
        change_df['time'] = pd.to_datetime(change_df[['year', 'month', 'day', 'hour', 'minute', 'second']])
        change_df['time'].drop_duplicates(keep='first', inplace=True)

        end_point = change_df.iloc[-1:].copy()
        end_point['day'] = end_point['day'] + 1
        end_point['hour'] = 0
        end_point['minute'] = 0
        end_point['second'] = 0
        end_point['time'] = pd.to_datetime(end_point[['year', 'month', 'day', 'hour', 'minute', 'second']])

        change_df = pd.concat([change_df, end_point], axis=0).copy()
        change_df.reset_index(inplace=True, drop=True)
        day_change_idx = list(change_df[['year', 'month', 'day']].drop_duplicates().index)

        seg_lst = []

        for i in range(len(day_change_idx) - 1):
            seg_lst.append(change_df.iloc[day_change_idx[i]:day_change_idx[i + 1] + 1].copy())

        results = Parallel(n_jobs=1)(delayed(get_info)(seg) for seg in tqdm(seg_lst))
        rslt = pd.concat(results)

        rslt.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second']).reset_index(drop=True).to_csv(
            f'Database/{number}.csv')

    except Exception as e:
        to_do_lst.append(number)
        print(f"An error occurred: {e}")

51969


  0%|          | 0/565 [00:00<?, ?it/s]

An error occurred: Can't pickle <function process_epoch at 0x000002262F6C6D40>: attribute lookup process_epoch on __main__ failed
58722



  0%|          | 0/200 [00:00<?, ?it/s]

An error occurred: Can't pickle <function process_epoch at 0x000002262F6C6D40>: attribute lookup process_epoch on __main__ failed
56361




KeyboardInterrupt



In [27]:
to_do_lst

[51969,
 58722,
 56361,
 56289,
 56783,
 48018,
 43823,
 45246,
 29349,
 37265,
 42691,
 42984,
 55841]

In [6]:
response = get_tle(number)
tle_datas = response.text.split('\r\n')

tle_first_lst = []
tle_second_lst = []

for i in range(0, len(tle_datas) - 2, 3):
    tle_first_lst.append(tle_datas[i + 1])
    tle_second_lst.append(tle_datas[i + 2])

tle_df = pd.DataFrame({'first_line': tle_first_lst, 'second_line': tle_second_lst})
tle_df.drop_duplicates(keep='first', inplace=True)
tle_df.reset_index(inplace=True, drop=True)

In [7]:
change_times = []
for i in range(len(tle_df)):
    tle_1, tle_2 = tle_df['first_line'].iloc[i], tle_df['first_line'].iloc[i]
    satellite = Satrec.twoline2rv(tle_1, tle_2)
    jdsatepoch = satellite.jdsatepoch
    jdsatepochfrac = satellite.jdsatepochF
    epochdatetime = invjday(jdsatepoch + jdsatepochfrac)
    change_times.append(epochdatetime)

In [8]:
change_df = pd.DataFrame(change_times)
change_df.columns = ['year', 'month', 'day', 'hour', 'minute', 'second']
change_df['first_line'] = tle_df['first_line']
change_df['second_line'] = tle_df['second_line']
change_df['time'] = pd.to_datetime(change_df[['year', 'month', 'day', 'hour', 'minute', 'second']])
change_df['time'].drop_duplicates(keep='first', inplace=True)

In [ ]:
from sgp4.api import Satrec
from sgp4.api import accelerated
import numpy as np

# 위성 객체 생성
satellite = Satrec.twoline2rv(line1, line2)

# 시간 배열 생성 (예: 0부터 24시간까지 1시간 간격)
jd_start = 2459270.5  # Julian date 시작점 (예제)
jd_fracts = np.linspace(0.0, 1.0, num=24)  # 0부터 1일까지 24개의 균등한 간격 생성

# 위치 및 속도 계산
e, r, v = accelerated.sgp4_array(satellite, jd_start, jd_fracts)

# 결과 출력
for i in range(len(jd_fracts)):
    print(f"시간 {jd_fracts[i]*24:.2f} 시간 후:")
    print(f"위치 (km): {r[i]}")
    print(f"속도 (km/s): {v[i]}")


In [10]:
def get_info(seg):
    change_df_seg = seg

    start_info = change_df_seg.iloc[0]
    epoch = datetime(start_info['year'], start_info['month'], start_info['day'], start_info['hour'],
                     start_info['minute'], int(start_info['second']))
    epoch += timedelta(minutes=1)

    end_info = change_df_seg.iloc[-1]
    end_epoch = datetime(end_info['year'], end_info['month'], end_info['day'], end_info['hour'], end_info['minute'],
                         int(end_info['second']))

    all_info = []

    rad2deg = 180.0 / np.pi
    earth_radius = 6378.137

    while epoch <= end_epoch:
        # print(epoch)
        year = epoch.year
        month = epoch.month
        date = epoch.day
        hour = epoch.hour
        minute = epoch.minute
        second = epoch.second

        now_time = f'{year}-{month}-{date} {hour}:{minute}:{second}'
        temp = change_df[change_df['time'] <= pd.to_datetime(now_time)].copy()
        tle1, tle2 = temp.iloc[-1]['first_line'], temp.iloc[-1]['second_line']

        satellite = Satrec.twoline2rv(tle1, tle2)
        jd, fr = jday(year, month, date, hour, minute, second)
        e, r, v = accelerated.sgp4_array(satellite, jd_start, jd_fracts)
        print(jday(end_epoch.year, end_epoch.month, end_epoch.date, end_epoch.hour, end_epoch.minute, end_epoch.second))
        
        end_epoch
        e, position, velocity = satellite.sgp4(jd, fr)

        # 추가된 부분
        ts = load.timescale()
        t = ts.utc(year, month, date, hour, minute, second)
        earth_satellite = EarthSatellite(tle1, tle2, '', ts)
        geocentric = earth_satellite.at(t)
        subpoint = wgs84.subpoint(geocentric)
        latitude = subpoint.latitude.degrees
        longitude = subpoint.longitude.degrees

        all_info.append([
            len(temp), tle1, tle2,
            epoch.year, epoch.month, epoch.day, epoch.hour, epoch.minute, epoch.second,
            *position, *velocity,
            (sum(p ** 2 for p in
                 position) ** 0.5) - earth_radius,
            (sum(v ** 2 for v in velocity) ** 0.5) / earth_radius,
            longitude,
            satellite.alta * earth_radius,
            satellite.altp * earth_radius,
            satellite.inclo * rad2deg, satellite.ecco,
            satellite.nodeo
        ])

        epoch += timedelta(minutes=1)

    info_df = pd.DataFrame(all_info)

    info_df.columns = ['change', 'tle1', 'tle2',
                       'year', 'month', 'day', 'hour', 'minute', 'second',
                       'x', 'y', 'z', 'vx', 'vy', 'vz',
                       'altitude', 'velocity', 'longitude',
                       'apogee', 'perigee',
                       'inclination', 'eccentricity', 'raan']

    return info_df

In [18]:
from sgp4.api import Satrec
from sgp4.api import accelerated
import numpy as np

epoch = datetime(2023, 1, 1, 1, 0, 0, 0)
all_info = []
earth_radius = 6378.137
rad2deg = 180.0 / np.pi
end_epoch =datetime(2024, 8, 1, 0, 0, 0, 0)
while epoch <= end_epoch:


    year = epoch.year
    month = epoch.month
    date = epoch.day
    hour = epoch.hour
    minute = epoch.minute
    second = epoch.second

    end_year = end_epoch.year
    end_month = end_epoch.month
    end_date = end_epoch.day
    end_hour = end_epoch.hour
    end_minute = end_epoch.minute
    end_second = end_epoch.second

    now_time = f'{year}-{month}-{date} {hour}:{minute}:{second}'
    temp = change_df[change_df['time'] <= pd.to_datetime(now_time)].copy()
    tle1, tle2 = temp.iloc[-1]['first_line'], temp.iloc[-1]['second_line']

    satellite = Satrec.twoline2rv(tle1, tle2)
    jd, fr = jday(year, month, date, hour, minute, second)

    end_jd, end_fr = jday(end_year, end_month, end_date, end_hour, end_minute, end_second)
    jd_fracts = np.linspace(jd+fr, end_jd+end_fr, )
    e, r, v = accelerated.sgp4_array(satellite, jd+fr, fr)
    print(r)

    e, position, velocity = satellite.sgp4(jd, fr)

    # 현재 시간 설정
    ts = load.timescale()
    t = ts.utc(year, month, date, hour, minute, 0)

    # EarthSatellite 객체 생성
    earth_satellite = EarthSatellite(tle1, tle2, '', ts)

    # 위성의 지구 중심 좌표 및 속도 계산
    geocentric = earth_satellite.at(t)

    # 위성의 위치를 지구의 위도와 경도로 변환
    subpoint = wgs84.subpoint(geocentric)

    # 위도, 경도 및 고도 추출
    latitude = subpoint.latitude.degrees
    longitude = subpoint.longitude.degrees

    print(f"Earth Radius at Location ({latitude}, {longitude})")

    all_info.append([
        len(temp), tle1, tle2,
        epoch.year, epoch.month, epoch.day, epoch.hour, epoch.minute, epoch.second,
        *position, *velocity,
        (sum(p ** 2 for p in position) ** 0.5) - earth_radius,
        (sum(v ** 2 for v in velocity) ** 0.5),
        longitude,
        satellite.alta * earth_radius,
        satellite.altp * earth_radius,
        satellite.inclo * rad2deg, 
        satellite.ecco,
        satellite.nodeo
    ])

    epoch += timedelta(minutes=1)

AttributeError: 'bool' object has no attribute 'sgp4_array'

In [76]:
info_df = pd.DataFrame(all_info)
info_df.columns = ['change', 'tle1', 'tle2', 'year', 'month', 'day', 'hour', 'minute', 'second',
                   'x', 'y', 'z', 'vx', 'vy', 'vz',
                   'altitude', 'velocity', 'longitude', 'apogee', 'perigee', 'inclination', 'eccentricity', 'raan']
info_df['change'] = (info_df['change'].diff().fillna(1) >= 1) * 1

# 그래프

In [44]:
from astropy.coordinates import TEME, CartesianDifferential, CartesianRepresentation
from astropy import units as u
from astropy.coordinates import ITRS
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

epoch = datetime(2023, 1, 1, 1, 0, 0, 0)
all_info = []

rad2deg = 180.0 / np.pi

while epoch < datetime(2024, 7, 31, 0, 0, 0, 0):
    print(epoch)
    year = epoch.year
    month = epoch.month
    date = epoch.day
    hour = epoch.hour
    minute = epoch.minute
    second = epoch.second

    now_time = f'{year}-{month}-{date} {hour}:{minute}:{second}'
    temp = change_df[change_df['time'] <= pd.to_datetime(now_time)].copy()
    tle1, tle2 = temp.iloc[-1]['first_line'], temp.iloc[-1]['second_line']

    satellite = Satrec.twoline2rv(tle1, tle2)
    jd, fr = jday(year, month, date, hour, minute, 0)
    e, temp_p, temp_v = satellite.sgp4(jd, fr)
    t = Time(jd + fr, format='jd')

    e, position, velocity = satellite.sgp4(t.jd1, t.jd2)
    position = CartesianRepresentation(position * u.km)
    velocity = CartesianDifferential(velocity * u.km / u.s)
    teme = TEME(position.with_differentials(velocity), obstime=t)
    itrs_geo = teme.transform_to(ITRS(obstime=t))
    location = itrs_geo.earth_location
    print(location.geodetic)

    distance = (sum(p ** 2 for p in temp_p) ** 0.5)
    height = location.geodetic.height.to_value(u.km)
    earth_radius = distance - height
    print(earth_radius)

    all_info.append([
        len(temp), tle1, tle2,
        epoch.year, epoch.month, epoch.day, epoch.hour, epoch.minute, epoch.second,
        *temp_p, *temp_v,
        height,
        (sum(v ** 2 for v in temp_v) ** 0.5),
        (sum(v ** 2 for v in temp_v) ** 0.5) / earth_radius,
        satellite.alta * earth_radius,
        satellite.altp * earth_radius,
        satellite.inclo * rad2deg, satellite.ecco,
        satellite.nodeo
    ])

    epoch += timedelta(minutes=1)

2023-01-01 01:00:00
GeodeticLocation(lon=<Longitude 71.56468161 deg>, lat=<Latitude 16.50865003 deg>, height=<Quantity 555.12127433 km>)
6376.422679805846
2023-01-01 01:01:00
GeodeticLocation(lon=<Longitude 70.76008809 deg>, lat=<Latitude 20.25269176 deg>, height=<Quantity 555.1993008 km>)
6375.5921907675565
2023-01-01 01:02:00
GeodeticLocation(lon=<Longitude 69.92827253 deg>, lat=<Latitude 23.99355628 deg>, height=<Quantity 555.40761556 km>)
6374.624142039359
2023-01-01 01:03:00
GeodeticLocation(lon=<Longitude 69.06187207 deg>, lat=<Latitude 27.7304376 deg>, height=<Quantity 555.73455605 km>)
6373.535316739776
2023-01-01 01:04:00
GeodeticLocation(lon=<Longitude 68.15200952 deg>, lat=<Latitude 31.46247373 deg>, height=<Quantity 556.16661141 km>)
6372.344591621122
2023-01-01 01:05:00
GeodeticLocation(lon=<Longitude 67.18774019 deg>, lat=<Latitude 35.18870335 deg>, height=<Quantity 556.68869817 km>)
6371.072602589798
2023-01-01 01:06:00
GeodeticLocation(lon=<Longitude 66.15526934 deg>, l


KeyboardInterrupt

